In [ ]:
from flask import Flask, request, jsonify, render_template
import numpy as np
import tensorflow as tf
import joblib

app = Flask(__name__, template_folder='templates')  # Setting up Flask app

# Loading your trained model and scaler
model = tf.keras.models.load_model('instagram_model.keras')  # Loading the trained model
scaler_x = joblib.load('scaler.pkl')  # Loading the scaler

@app.route('/')
def index():
    return render_template('index.html')  # The HTML template for the homepage

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Getting input data from the frontend (JSON format)

    # Extracting features from the JSON request
    features = [
        data['profile_pic'],
        data['username_length'],
        data['fullname_words'],
        data['fullname_length'],
        data['description_length'],
        data['external_url'],
        data['private'],
        data['posts'],
        data['nums_length_fullname'],
        data['followers'],
        data['follows']
    ]

    # Converting features to a numpy array and reshape to match model input
    features = np.array(features).reshape(1, -1)

    # Scaling the features using the same scaler as during training
    scaled_features = scaler_x.transform(features)

    # Predicting using the model
    prediction = model.predict(scaled_features)

    # Getting the predicted class (0 for 'real', 1 for 'fake')
    predicted_class = np.argmax(prediction)

    # Returning the result in JSON format
    result = {'prediction': 'REAL' if predicted_class == 0 else 'FAKE'}
    return jsonify(result)

# Function to run Flask in a separate thread
def run_app():
    app.run(debug=True, use_reloader=False, host='127.0.0.1', port=5000)

# Start Flask in a new thread to avoid blocking
if __name__ == "__main__":
    from threading import Thread
    thread = Thread(target=run_app)
    thread.start()
